In [1]:
import gym
import torch
import numpy as np
import sys
from torchvision.transforms import ToTensor, Lambda
from torch import nn
from my_model import Network, policy
from utils import dataset, dataloader, buffer
import trainer
import random
import torch.onnx as onnx
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('RLresult/cartpole/dqn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'


/home/jongsu/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
env = gym.make('CartPole-v1') 
print("act", env.action_space) # 0 or 1
print("obs", env.observation_space) #size 4 arry

act Discrete(2)
obs Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [3]:
env.observation_space.sample()

array([ 2.4903133e+00,  2.1446022e+38, -7.5590163e-02, -2.0055699e+38],
      dtype=float32)

In [4]:
HIDDEN_SIZE = 32
BATCH_SIZE = 10000
learning_rate = 0.01
CART_ACTION = 2
CAPACITY = 10000
TRAIN_ITER = CAPACITY*10/BATCH_SIZE
PARAM_PATH = 'DQN_param/cartpoleDQN.pth'
PARAM_PATH_TEST = 'DQN_param/cartpoleDQN_test.pth'
#print(Transition(2,1,2,3))
#Transition(state=2, action=1, next_state=2, reward=3)

In [5]:
input_state_size = len(env.observation_space.sample())
updatedDQN = Network.SimpleNN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)
baseDQN = Network.SimpleNN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)

In [6]:
updatedDQN.load_state_dict(torch.load(PARAM_PATH_TEST))
baseDQN.load_state_dict(updatedDQN.state_dict())
baseDQN.eval()

SimpleNN(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
    (5): ReLU()
  )
)

In [7]:
DQN_policy = policy.Policy("DQN", baseDQN, "cart")

mybuffer = buffer.Simulate(env, DQN_policy)
mycartdata = dataset.SimData(capacity = CAPACITY)
mybuffer.renewal_memory(CAPACITY, mycartdata)



In [8]:
%%time

mydataloader = dataloader.CustomDataLoader(mycartdata, batch_size=BATCH_SIZE)
mytrain = trainer.Train("cart", mydataloader, "DQN")

CPU times: user 37 µs, sys: 5 µs, total: 42 µs
Wall time: 42.9 µs


In [9]:
optimizer = torch.optim.SGD(updatedDQN.parameters(), lr=learning_rate)
train_iter = 100
i = 0
while(i < train_iter):
    i = i + 1
    mybuffer.renewal_memory(CAPACITY, mycartdata)
    loss = mytrain.training(TRAIN_ITER, BATCH_SIZE, optimizer, updatedDQN, baseDQN)
    writer.add_scalar("loss", loss, i)
    torch.save(updatedDQN.state_dict(), PARAM_PATH_TEST)
    baseDQN.load_state_dict(updatedDQN.state_dict())
    baseDQN.eval()
    
env.close()

loss =  tensor(0.8003, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(0.8352, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.0046, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.0933, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.1690, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.2363, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.1758, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.2172, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.2788, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(1.1082, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(0.9391, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(0.8225, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(0.6376, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(0.5049, device='cuda:0', grad_fn=<MseLossBackward0>)
loss =  tensor(0.4444, device='cuda:0', grad_fn=

KeyboardInterrupt: 

In [ ]:
writer.flush()
writer.close()